# Get started

In [ ]:
# Imports
import os
from shutil import copy2
import urllib.request
from zipfile import ZipFile
from pathlib import Path

from sepal_ui import mapping as sm
from ipyleaflet import GeoJSON
import geopandas as gpd
from sepal_ui import color
import rasterio as rio
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
import numpy as np

import forestatrisk as far

home = str(Path.home())

We create a directory to hold the outputs with the help of the function `.make_dir()`.

In [ ]:
# Make output directory
output_path = f"{home}/far/output"
far.make_dir(output_path)

## 1. Data

### 1.1 Import and unzip the data

We use the [Guadeloupe](https://en.wikipedia.org/wiki/Guadeloupe) archipelago as a case study.

In [ ]:
# Source of the data
url = (
    "https://github.com/ghislainv/forestatrisk/raw/master/docsrc/notebooks/data_GLP.zip"
)
zip_path = f"{home}/tmp/data_GLP.zip"
data_path = f"{home}/far/data"

if os.path.exists(zip_path) is False:
    urllib.request.urlretrieve(url, zip_path)

with ZipFile(zip_path, "r") as z:
    z.extractall(data_path)

### 1.2 Files

The `data` folder includes, among other files:

- The forest cover change data for the period 2010-2020 as a GeoTiff raster file (`data/fcc23.tif`).
- Spatial variables as GeoTiff raster files (`.tif` extension, eg. `data/dist_edge.tif` for distance to forest edge).

### 1.3 Sampling the observations



In [ ]:
# Sample points
dataset = far.sample(
    nsamp=10000,
    adapt=True,
    seed=1234,
    csize=10,
    var_dir=data_path,
    input_forest_raster="fcc23.tif",
    output_file=f"{output_path}/sample.txt",
    blk_rows=0,
)

In [ ]:
# Remove NA from data-set (otherwise scale() and
# model_binomial_iCAR do not work)
dataset = dataset.dropna(axis=0)
# Set number of trials to one for far.model_binomial_iCAR()
dataset["trial"] = 1
# Print the first five rows
dataset.head(5)

## 2. Model

### 2.1 Model preparation

In [ ]:
# Neighborhood for spatial-autocorrelation
nneigh, adj = far.cellneigh(raster=f"{data_path}/fcc23.tif", csize=10, rank=1)

# List of variables
variables = [
    "scale(altitude)",
    "scale(slope)",
    "scale(dist_defor)",
    "scale(dist_edge)",
    "scale(dist_road)",
    "scale(dist_town)",
    "scale(dist_river)",
]

# Formula
right_part = " + ".join(variables) + " + cell"
left_part = "I(1-fcc23) + trial ~ "
formula = left_part + right_part

# Starting values
beta_start = -99  # Simple GLM estimates

# Priors
priorVrho = -1  # -1="1/Gamma"

### 2.2 iCAR model

In [ ]:
# Run the model
mod_binomial_iCAR = far.model_binomial_iCAR(
    suitability_formula=formula,
    data=dataset,  # Observations
    n_neighbors=nneigh,
    neighbors=adj,  # Spatial structure
    priorVrho=priorVrho,  # Priors
    burnin=1000,
    mcmc=1000,
    thin=1,  # Chains
    beta_start=beta_start,
)  # Starting values

### 2.3 Model summary

In [ ]:
# Predictions
pred_icar = mod_binomial_iCAR.theta_pred

# Summary
display(mod_binomial_iCAR)

# Write summary in file
with open(f"{output_path}/summary_icar.txt", "w") as f:
    f.write(str(mod_binomial_iCAR))

## 3. Predict

### 3.1 Interpolate spatial random effects

In [ ]:
# Spatial random effects
rho = mod_binomial_iCAR.rho

# Interpolate
far.interpolate_rho(
    rho=rho,
    input_raster=f"{data_path}/fcc23.tif",
    output_file=f"{output_path}/rho.tif",
    csize_orig=10,
    csize_new=1,
)

### 3.2 Predict deforestation probability

In [ ]:
# Update dist_edge and dist_defor at t3
os.rename(f"{data_path}/dist_edge.tif", f"{data_path}/dist_edge.tif.bak")
os.rename(f"{data_path}/dist_defor.tif", f"{data_path}/dist_defor.tif.bak")
copy2(f"{data_path}/forecast/dist_edge_forecast.tif", f"{data_path}/dist_edge.tif")
copy2(f"{data_path}/forecast/dist_defor_forecast.tif", f"{data_path}/dist_defor.tif")

# Compute predictions
far.predict_raster_binomial_iCAR(
    mod_binomial_iCAR,
    var_dir=f"{data_path}",
    input_cell_raster=f"{output_path}/rho.tif",
    input_forest_raster=f"{data_path}/forest/forest_t3.tif",
    output_file=f"{output_path}/prob.tif",
    blk_rows=10,  # Reduced number of lines to avoid memory problems
)

# Reinitialize data
os.remove(f"{data_path}/dist_edge.tif")
os.remove(f"{data_path}/dist_defor.tif")
os.rename(f"{data_path}/dist_edge.tif.bak", f"{data_path}/dist_edge.tif")
os.rename(f"{data_path}/dist_defor.tif.bak", f"{data_path}/dist_defor.tif")

## 4. Project future forest cover change

In [ ]:
# Forest cover
fc = list()
dates = ["t2", "t3"]
ndates = len(dates)
for i in range(ndates):
    rast = f"{data_path}/forest/forest_" + dates[i] + ".tif"
    val = far.countpix(input_raster=rast, value=1)
    fc.append(val["area"])  # area in ha
# Save results to disk
f = open(f"{output_path}/forest_cover.txt", "w")
for i in fc:
    f.write(str(i) + "\n")
f.close()
# Annual deforestation
T = 10.0
annual_defor = (fc[0] - fc[1]) / T
print(
    "Mean annual deforested area during the period 2010-2020: {} ha/yr".format(
        annual_defor
    )
)

In [ ]:
# Projected deforestation (ha) during 2020-2050
defor = annual_defor * 30

# Compute future forest cover in 2050
stats = far.deforest(
    input_raster=f"{output_path}/prob.tif",
    hectares=defor,
    output_file=f"{output_path}/fcc_2050.tif",
    blk_rows=128,
)

## 5. Figures

### 5.0 Borders

Init the map and the border of the aoi 

In [ ]:
# create the map
map_ = sm.SepalMap()

# read the borders
border_file = f"{data_path}/ctry_PROJ.shp"
border_gdf = gpd.read_file(border_file).to_crs(4326)

border_layer = GeoJSON(
    data=border_gdf.__geo_interface__,
    style={
        "stroke": True,
        "color": color.primary,
        "weight": 2,
        "opacity": 1,
        "fill": False,
    },
    name="borders",
)

map_.add_layer(border_layer)
map_.zoom_bounds(border_gdf.total_bounds);

### 5.1 Historical forest cover change

Forest cover change for the period 2000-2010-2020

In [ ]:
# Plot forest
cmax = 255.0  # float for division
colors = [
    (1, 1, 1, 0),  # transparent for 0
    (255, 165, 0, 255),  # green
    (227, 26, 28, 255),  # orange
    (34, 139, 34, 255),  # red
]

map_.add_raster(
    f"{data_path}/forest/fcc123.tif",
    layer_name="forest cover 123",
    colormap=ListedColormap(
        [tuple([v / cmax for v in c]) for i, c in enumerate(colors)]
    ),
    colorbar_position=False,
    fit_bounds=False,
)

### 5.2 Spatial random effects

In [ ]:
# Original spatial random effects
map_.add_raster(
    f"{output_path}/rho_orig.tif",
    layer_name="original spatial random effect",
    colormap="RdYlGn_r",
    colorbar_position=False,
    fit_bounds=False,
)

# Interpolated spatial random effects
map_.add_raster(
    f"{output_path}/rho.tif",
    layer_name="Interpolated spatial random effects",
    colormap="RdYlGn_r",
    colorbar_position=False,
    fit_bounds=False,
)

### 5.3 Spatial probability of deforestation

In [ ]:
# create a new colormap
# Colormap
colors = []
cmax = 255.0  # float for division
vmax = 65535.0  # float for division
colors.append((0, (34 / cmax, 139 / cmax, 34 / cmax, 1)))  # green
colors.append((1 / vmax, (34 / cmax, 139 / cmax, 34 / cmax, 1)))  # green
colors.append((39322 / vmax, (1, 165 / cmax, 0, 1)))  # orange, p=0.60
colors.append((52429 / vmax, (227 / cmax, 26 / cmax, 28 / cmax, 1)))  # red, p=0.80
colors.append((1, (0, 0, 0, 1)))  # black
color_map = LinearSegmentedColormap.from_list(
    name="mycm", colors=colors, N=65535, gamma=1.0
)

# transparent, must be associated with vmin
color_map.set_under(color=(1, 1, 1, 0))

# Spatial probability of deforestation
map_.add_raster(
    f"{output_path}/prob.tif",
    layer_name="Spatial probability of deforestation",
    colormap=color_map,
    colorbar_position=False,
    fit_bounds=False,
)

### 5.4 Future forest cover

In [ ]:
# create the colormap
colors = []
cmax = 255.0  # float for division
col_defor = tuple(np.array([34, 139, 34, 255]) / cmax)
col_for = tuple(np.array([227, 26, 28, 255]) / cmax)
colors.append((0, 0, 0, 0))  # transparent
colors.append(col_for)
colors.append(col_defor)
color_map = ListedColormap(colors)

# Projected forest cover change (2020-2050)
map_.add_raster(
    f"{output_path}/fcc_2050.tif",
    layer_name="Projected forest cover change (2020-2050)",
    colormap=color_map,
    colorbar_position=False,
    fit_bounds=False,
)

In [ ]:
map_